# Turkish railways - pop and ntl

Compare population and nighttime lights around existing, under-construction, and proposed railways in Turkey

In [1]:
import os, sys
import requests
import fiona

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx

sys.path.insert(0, "C:/WBG/Work/Code/GOSTrocks/src")
import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.rasterMisc as rMisc

from GOSTrocks.misc import tPrint
from space2stats_client import Space2StatsClient
from shapely import from_geojson


requests.packages.urllib3.disable_warnings() 

client = Space2StatsClient(verify_ssl=False)

%load_ext autoreload
%autoreload 2 

In [2]:
base_folder = r"C:\WBG\Work\Projects\TUR_Railways"
railway_folder = os.path.join(base_folder, "Data", "Source")
results_folder = os.path.join(base_folder, "Data", "Results")

projected_railways = os.path.join(railway_folder, "Projected Railways", "doc.kml")
under_construction_railways = os.path.join(railway_folder, "Railways Under Construction", "doc.kml")
existing_railways = os.path.join(railway_folder, "existing", "turrail.shp")

pop_file = os.path.join(base_folder, "Data", "tur_ppp_2020_UNadj_constrained.tif")
admin_bounds = os.path.join(base_folder, "Data", "TUR_ADM_1.gpkg")
metro_def_file = os.path.join(base_folder, "Data", "TUR_ADM1.csv")

In [3]:
ISO3 = "TUR" # Turkey
ADM = "ADM2" # Level 2 administrative boundaries
m_crs = 5636
adm_boundaries = client.fetch_admin_boundaries(ISO3, ADM)
adm1_boundaries = gpd.read_file(admin_bounds)
metro_def = pd.read_csv(metro_def_file)
adm1_boundaries = pd.merge(adm1_boundaries, metro_def.loc[:,["ADM1CD_c", "Metropolitan"]], on="ADM1CD_c")
national_bounds = client.fetch_admin_boundaries(ISO3, "ADM0")

# List all S2S topics
topics = client.get_topics()
topics

Item ID,name,description,source_data
space2stats_population_2020,Population,Gridded population disaggregated by gender.,"WorldPop gridded population, 2020, Unconstrain..."
flood_exposure_15cm_1in100,Population Exposed to Floods,Population where flood depth is greater than 1...,Fathom 3.0 High Resolution Global Flood Maps I...
urbanization_ghssmod,Urbanization by population and by area,Urbanization is analyzed using the GHS-SMOD da...,Global Human Settlement Layer (https://human-s...
nighttime_lights,Nighttime Lights,Sum of luminosity values measured by monthly c...,"World Bank - Light Every Night, https://regist..."
climate,Standardized Precipitation Index (SPI),Index for a given timescale measuring drought ...,CHIRPS3
builtarea_ghsl,Built area,Built area (in m2) in 5-year epochs. Source da...,https://human-settlement.emergency.copernicus....


In [4]:
# Download the nighttime lights values for the AOI
properties = client.get_properties("nighttime_lights")
sel_fields = list(properties['name'].values[:-1])
df = client.get_summary(
    gdf=adm1_boundaries,                     # Area of Interest
    spatial_join_method="centroid",         # Spatial join method (between h3 cells and each feature)
    fields=sel_fields,                      # Fields from Space2Stats to query
    geometry="polygon"                      # Whether to return the geometry of the hexagons
)

df["geometry"] = df["geometry"].apply(lambda geom: from_geojson(geom))

# Convert dataframe to GeoDataFrame
tur_s2s = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
tur_s2s.head()

Fetching data for boundary 1 of 80...
Fetching data for boundary 2 of 80...
Fetching data for boundary 3 of 80...
Fetching data for boundary 4 of 80...
Fetching data for boundary 5 of 80...
Fetching data for boundary 6 of 80...
Fetching data for boundary 7 of 80...
Fetching data for boundary 8 of 80...
Fetching data for boundary 9 of 80...
Fetching data for boundary 10 of 80...
Fetching data for boundary 11 of 80...
Fetching data for boundary 12 of 80...
Fetching data for boundary 13 of 80...
Fetching data for boundary 14 of 80...
Fetching data for boundary 15 of 80...
Fetching data for boundary 16 of 80...
Fetching data for boundary 17 of 80...
Fetching data for boundary 18 of 80...
Fetching data for boundary 19 of 80...
Fetching data for boundary 20 of 80...
Fetching data for boundary 21 of 80...
Fetching data for boundary 22 of 80...
Fetching data for boundary 23 of 80...
Fetching data for boundary 24 of 80...
Fetching data for boundary 25 of 80...
Fetching data for boundary 26 of 8

,ISO_A3,ISO_A2,WB_A3,HASC_0,HASC_1,GAUL_0,GAUL_1,WB_REGION,WB_STATUS,SOVEREIGN,...,sum_viirs_ntl_2015,sum_viirs_ntl_2016,sum_viirs_ntl_2017,sum_viirs_ntl_2018,sum_viirs_ntl_2019,sum_viirs_ntl_2020,sum_viirs_ntl_2021,sum_viirs_ntl_2022,sum_viirs_ntl_2023,sum_viirs_ntl_2024
0,TUR,TR,TUR,TR,TR.AV,249,3026,ECA,Member State,TUR,...,179.200012,297.839996,257.440002,183.550018,241.480011,285.760010,331.869995,288.160034,346.290009,297.619995
1,TUR,TR,TUR,TR,TR.AV,249,3026,ECA,Member State,TUR,...,192.989990,436.459991,287.709991,233.379990,320.510010,395.299988,330.269989,314.760010,366.470001,327.230011
2,TUR,TR,TUR,TR,TR.AV,249,3026,ECA,Member State,TUR,...,235.299988,381.230011,322.149994,279.820007,396.690002,443.660004,392.470001,444.039978,507.309998,458.549988
3,TUR,TR,TUR,TR,TR.AV,249,3026,ECA,Member State,TUR,...,203.569992,285.660004,253.250000,135.119995,239.789993,283.209991,217.059998,285.390015,329.330017,186.710007
4,TUR,TR,TUR,TR,TR.AV,249,3026,ECA,Member State,TUR,...,893.239990,1747.179932,1107.069946,970.840088,1150.010010,1144.910034,1221.699951,1075.979980,1422.750000,1259.769897


In [13]:
from h3ronpy.pandas.vector import geodataframe_to_cells, cells_dataframe_to_geodataframe
from h3ronpy import ContainmentMode

def get_bounds(in_shp, gID, h3_lvl=6):
    """ Generate a geodataframe for the supplied in_shp with the H3 cells and % overlap

    Parameters
    ----------
    in_shp : geopandas.GeoDataFrame
        The input shapely polygon as a geopandas dataframe
    gID : string
        The column name to use for the ID in the output
    h3_lvl : int
        The H3 level to use for the hexagons, default is 6
    """
    # extract the H3 cells
    cols_to_keep = [gID, 'cell', 'overlap']
    cell_ax = cells_dataframe_to_geodataframe(geodataframe_to_cells(in_shp, 6, ContainmentMode.IntersectsBoundary))
    cell_ax['cell'] = cell_ax['cell'].apply(lambda x: hex(x)[2:])    
    # Identify contained and overlapping hexes with the admin bounds
    contained_h3 = cell_ax.sjoin(in_shp, predicate='within')
    missed_h3 = cell_ax[~cell_ax['cell'].isin(contained_h3['cell'])]
    # calculate h3x overlap with feature
    shp_area = in_shp.union_all()
    cell_ax['overlap'] = 0.0
    cell_ax.loc[contained_h3.index, 'overlap'] = 1.0
    cell_ax.loc[missed_h3.index, 'overlap'] = cell_ax.loc[missed_h3.index,'geometry'].apply(lambda x: x.intersection(shp_area).area/x.area)\
    
    return cell_ax.loc[:, cols_to_keep].reset_index(drop=True)

In [17]:
geodataframe_to_cells?

Signature:
geodataframe_to_cells(
    gdf: geopandas.geodataframe.GeoDataFrame,
    resolution: int,
    containment_mode: ContainmentMode = ContainmentMode.ContainsCentroid,
    compact: bool = False,
    cell_column_name: str = 'cell',
) -> pandas.core.frame.DataFrame
Docstring:
Convert a `GeoDataFrame` to H3 cells while exploding all other columns according to the number of cells derived
from the rows geometry.

The conversion of GeoDataFrames is parallelized using the available CPUs.

The duplication of all non-cell columns leads to increased memory requirements. Depending on the use-case
some of the more low-level conversion functions should be preferred.

:param gdf:
:param resolution: H3 resolution
:param containment_mode: Containment mode used to decide if a cell is contained in a polygon or not.
        See the ContainmentMode class.
:param compact: Compact the returned cells by replacing cells with their parent cells when all children
        of that cell are part of the set.

In [14]:
# Calculate the total nighttime lights for each administrative area, based on overlap with the administrative boundaries
for lbl, curD in tur_s2s.groupby("ADM1CD_c"):
    cur_admin_boundary = adm1_boundaries[adm1_boundaries["ADM1CD_c"] == lbl]
    hex_overlap = get_bounds(cur_admin_boundary.union_all(), "ADM1CD_c")
    
    break

AttributeError: 'MultiPolygon' object has no attribute 'geometry'

In [ ]:
# The population values are not correct in S2S, update with specific input pop raster
res = rMisc.zonalStats(tur_s2s, pop_file, minVal=0, return_df=True)
tur_s2s['sum_pop_2020'] = res['SUM']
sel_fields = sel_fields + ['sum_pop_2020']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
tur_s2s.plot(ax=ax, column="sum_viirs_ntl_2012", 
         legend=True, cmap="Reds", alpha=0.75, 
         scheme="naturalbreaks", k=5, 
         legend_kwds=dict(title='NTL 2012', fmt="{:,.0f}"),
         linewidth=0)
ctx.add_basemap(ax, source=ctx.providers.Esri.WorldPhysical, verify=False)
plt.axis("off")
plt.show()


## Summarize nighttime lights and population within 15km of various railways

In [ ]:
# The KML/KMZ files from the source are garbage. In order to use the KML files, we need to read them and extract the layers oddly
from lxml import etree

def list_kml_layers(filename, method=1):
    tree = etree.parse(filename)
    root = tree.getroot()
    namespaces = {'kml': 'http://www.opengis.net/kml/2.2'}

    if method == 1:
        
        layers = root.findall(".//{kml}Document", namespaces)
        layer_names = []
        for layer in layers:
            name_element = layer.find('{kml}name', namespaces)
            if name_element is not None:
                layer_names.append(name_element.text)
            else:
                layer_names.append("Unnamed Layer")
        return layer_names
    else:
        layers = []
        for element in root.iter():
            if element.tag.endswith('Document') or element.tag.endswith('Folder'):
                name_element = element.find('kml:name', namespaces)
                layer_name = name_element.text if name_element is not None else "Unnamed Layer"
                layers.append(layer_name)
        return layers

def gpd_read_all_layers(filename):
    layers = list_kml_layers(filename, method=1)
    good_layers = []
    for cur_layer in layers:
        try:
            curD = gpd.read_file(filename, driver="KML", layer=cur_layer)
            print(f"Processing layer: {cur_layer} - {curD.union_all().geom_type}")
            if curD.union_all().geom_type in ["LineString", "MultiLineString"]:
                curD['Label'] = cur_layer
                good_layers.append(curD)
        except:
            print(f"Layer {cur_layer} not found or could not be read.")
    return pd.concat(good_layers, ignore_index=True)

In [ ]:
updated_projected_file = os.path.join(railway_folder, "projected_railways.gpkg")
updated_construction_file = os.path.join(railway_folder, "under_construction_railways.gpkg")
if not os.path.exists(updated_projected_file):
    projected_rail_gpd = gpd_read_all_layers(projected_railways)
    projected_rail_gpd.to_file(updated_projected_file, driver="GPKG")
else:
    projected_rail_gpd = gpd.read_file(updated_projected_file)
if not os.path.exists(updated_construction_file):
    under_construction_rail_gpd = gpd.read_file(under_construction_railways, driver="KML")    
else:
    under_construction_rail_gpd = gpd.read_file(updated_construction_file)

existing_rail_gpd = gpd.read_file(existing_railways)

In [ ]:
# Convert projected and under construction to single row GeoDataFrames
projected_rail_gpd = projected_rail_gpd.dissolve().reset_index(drop=True)
under_construction_rail_gpd = under_construction_rail_gpd.dissolve().explode().reset_index(drop=True)
under_construction_rail_gpd.to_file(updated_construction_file, driver="GPKG")

# The under_construction railways are present in the projected railways file, so we need to remove them
projected_rail_gpd['geometry'].iloc[0] = projected_rail_gpd['geometry'].iloc[0].difference(under_construction_rail_gpd['geometry'].iloc[0])

# Explode the geometries to have one row per segment
projected_rail_gpd = projected_rail_gpd.dissolve().explode().reset_index(drop=True)
projected_rail_gpd = projected_rail_gpd.to_crs(epsg=m_crs)
projected_rail_gpd['length'] = projected_rail_gpd['geometry'].length
projected_rail_gpd.sort_values(by='length')
projected_rail_gpd = projected_rail_gpd.loc[projected_rail_gpd['length'] > 10]
projected_rail_gpd.loc[projected_rail_gpd['length'] > 10].to_file(os.path.join(railway_folder, "projected_railways_long.gpkg"), driver="GPKG")

In [ ]:
def get_s2s_sums(in_shape, s2s, buffer_dist=15000):
    # read in railways and buffer
    in_shape_buffer = in_shape.to_crs(m_crs)  
    in_shape_buffer["geometry"] = in_shape_buffer.buffer(buffer_dist)
    all_shape = in_shape_buffer.union_all() 

    # Identify S2S hexagons that intersect with the buffered railways
    s2s_cols = s2s.columns.tolist()
    s2s = gpd.sjoin(s2s, in_shape_buffer, how="inner", predicate="intersects")
    s2s = s2s.drop_duplicates(subset=["hex_id"])
    s2s = s2s.loc[:, s2s_cols]  # Keep only the original S2S columns
    
    # determine S2S overlap with buffered railways
    s2s['overlap'] = s2s['geometry'].apply(lambda x: x.intersection(all_shape).area/x.area)
    combo_h3 = s2s.sjoin(in_shape_buffer, how="inner", predicate="intersects")
    combo_h3 = combo_h3.drop_duplicates(subset=["hex_id"])
    
    #Calculate sums based on overlap
    all_results = {}
    for col in sel_fields:
        cur_results = (combo_h3[col] * combo_h3['overlap']).sum()
        all_results[col] = cur_results
    return all_results

railway_results_file = os.path.join(results_folder, "railway_s2s_summary.csv")
all_rails = pd.concat([existing_rail_gpd,under_construction_rail_gpd], ignore_index=True)
if not os.path.exists(railway_results_file):
    if tur_s2s.crs != m_crs:
        tur_s2s = tur_s2s.to_crs(m_crs)
    tPrint("Processing railways and S2S")
    existing_res = get_s2s_sums(existing_rail_gpd, tur_s2s, buffer_dist=15000)    
    tPrint("Completed existing railways")
    #projected_res = get_s2s_sums(projected_rail_gpd.dissolve(), tur_s2s, buffer_dist=15000)
    tPrint("Completed projected railways")
    under_construction_res = get_s2s_sums(under_construction_rail_gpd.dissolve(), tur_s2s, buffer_dist=15000)
    tPrint("Completed under construction railways")
    all_res = get_s2s_sums(all_rails.dissolve(), tur_s2s, buffer_dist=15000)
    pd.DataFrame({
        "All Railways": all_res,
        "Existing Railways": existing_res,
     #   "Projected Railways": projected_res,
        "Under Construction Railways": under_construction_res
    }).T.to_csv(railway_results_file)

In [ ]:
railway_results_file

## Summarize effects on muncipalities

In [ ]:
adm1_boundaries = adm1_boundaries.to_crs(m_crs)
# run summaries on tur_s2s
all_res = {}
for lbl, df in tur_s2s.groupby("ADM1CD_c"):
    adm_shape = adm1_boundaries.loc[adm1_boundaries["ADM1CD_c"] == lbl, "geometry"].values[0]
    df['overlap'] = df['geometry'].apply(lambda x: x.intersection(adm_shape).area/x.area)
    results = {}
    for col in sel_fields + ['sum_pop_2020']:
        results[col] = (df[col] * df['overlap']).sum()
    all_res[lbl] = results

In [ ]:
pd.DataFrame(all_res).T

In [ ]:
adm_summaries = pd.merge(adm1_boundaries, pd.DataFrame(all_res).T, left_on="ADM1CD_c", right_index=True)
adm_summaries

In [ ]:
# identify the metropolitan areas that intersect railways
metro_adm = adm_summaries.loc[adm_summaries['Metropolitan'] == 1]

# project all layers to m_crs
projected_rail_gpd = projected_rail_gpd.to_crs(epsg=m_crs)
under_construction_rail_gpd = under_construction_rail_gpd.to_crs(epsg=m_crs)
existing_rail_gpd = existing_rail_gpd.to_crs(epsg=m_crs)
metro_adm = metro_adm.to_crs(epsg=m_crs)
metro_adm['geometry'] = metro_adm.buffer(15000)  # Buffer the metropolitan areas

# Check which metropolitan areas intersect projected railways
metro_intersections_pr = metro_adm.sjoin(projected_rail_gpd, how="inner", predicate="intersects")
metro_intersections_pr = metro_intersections_pr.drop_duplicates(subset=["ADM1CD_c"])
# Check with which metropolitan areas intersect under construction railways
metro_intersections_uc = metro_adm.sjoin(under_construction_rail_gpd, how="inner", predicate="intersects")
metro_intersections_uc = metro_intersections_uc.drop_duplicates(subset=["ADM1CD_c"])
# Check which metropolitan areas intersect existing railways
metro_intersections_ex = metro_adm.sjoin(existing_rail_gpd, how="inner", predicate="intersects")
metro_intersections_ex = metro_intersections_ex.drop_duplicates(subset=["ADM1CD_c"])

for cur_intersections, label in zip(
        [metro_intersections_pr, metro_intersections_uc, metro_intersections_ex],
        ["Projected Railways", "Under Construction Railways", "Existing Railways"]
    ):
    print(f"\n{label}, {cur_intersections.shape[0]}, {cur_intersections['sum_pop_2020'].sum()}")


# Misc functions

In [ ]:
# Download ntl layers
ntl_files = ntlMisc.generate_annual_composites(adm_boundaries, out_folder=os.path.join(base_folder, "Data", "NTL"))

In [ ]:
tur_s2s.to_file(os.path.join(results_folder, "tur_s2s.gpkg"), driver="GPKG")

In [ ]:
adm1_boundaries.to_file(admin_bounds.replace(".gpkg", "_metro.gpkg"), driver="GPKG")